### 1.  This code imports essential Python libraries for data analysis and web interaction: requests for making HTTP requests (useful for APIs), pandas for handling and analyzing tabular data, time for managing delays or timestamps, json for working with JSON data formats, and os for interacting with the operating system, such as file and path management.

In [10]:
import requests
import pandas as pd
import time
import json
import os

### 2. This code sets up and sends a GET request to the PlanetTerp API to retrieve a list of professors. It defines the API endpoint (`base_url`), sets up pagination parameters (`limit` and `offset`), and passes them as query parameters (`params`) in the request. The response from the API is stored in `response`, and `response.status_code` checks whether the request was successful (e.g., 200 means OK).


In [11]:
base_url = "https://planetterp.com/api/v1/professors"
all_reviews = []
limit = 100
offset = 0
params = {
            "limit": limit,
            "offset": offset
}
response = requests.get(base_url, params=params)
response.status_code

200

### 3. This code parses the JSON response from the API into a Python dictionary using `response.json()`, then prints it in a nicely formatted, indented layout using `json.dumps()`. This makes the data easier to read and inspect, especially when dealing with nested structures.


In [12]:
data = response.json()
print(json.dumps(data, indent=2))

[
  {
    "courses": [
      "INST737",
      "ENPM808W",
      "ENPM808W",
      "ENPM808W",
      "ENPM606"
    ],
    "average_rating": null,
    "type": "professor",
    "name": "A Seyed",
    "slug": "seyed"
  },
  {
    "courses": [
      "MUSC800W",
      "MUSC830W",
      "MUSC830W",
      "MUSC830W"
    ],
    "average_rating": null,
    "type": "professor",
    "name": "Aaron Goldman",
    "slug": "goldman_aaron"
  },
  {
    "courses": [
      "THET678",
      "THET499"
    ],
    "average_rating": null,
    "type": "professor",
    "name": "Aaron Posner",
    "slug": "posner"
  },
  {
    "courses": [
      "NFSC431",
      "NFSC679R",
      "NFSC112",
      "HLTH672",
      "HLTH710",
      "NFSC672",
      "NFSC112",
      "NFSC679R",
      "NFSC431",
      "NFSC112",
      "NFSC679R",
      "NFSC431",
      "NFSC112",
      "NFSC679R",
      "NFSC431"
    ],
    "average_rating": 3.6667,
    "type": "professor",
    "name": "Abani Pradhan",
    "slug": "pradhan"
  },
  {

### 4. This function, fetch_planetterp_reviews(), retrieves professor review data from the PlanetTerp API in batches using pagination, filters for reviews that include both a course and professor name, and saves the cleaned data to a CSV file (PT_review_data.csv). If the file already exists and contains data, it loads it directly instead of downloading again. The result is returned as a pandas DataFrame.mm

In [13]:
def fetch_planetterp_reviews():
    """
    Fetches professor review data from the PlanetTerp API,
    extracts relevant fields, and saves it to PT_review_data.csv.
    Returns a DataFrame.
    """
    filename = "PT_review_data.csv"

    # Use cached file if it exists and is not empty
    if os.path.exists(filename) and os.path.getsize(filename) > 0:
        print("Loaded existing review data.")
        return pd.read_csv(filename)

    print("Downloading review data from PlanetTerp API...")
    base_url = "https://planetterp.com/api/v1/professors"
    offset = 0
    limit = 100
    reviews = []
    count = 0

    while True:
        try:
            response = requests.get(base_url, params={"limit": limit,"reviews": True, "offset": offset})
            if response.status_code != 200:
                print(f"HTTP error {response.status_code} at offset {offset}")
                return pd.DataFrame()
            data = response.json()
        except Exception as e:
            print(f"Request or JSON error at offset {offset}: {e}")
            return pd.DataFrame()

        if not data:
            break

        for prof in data:
            if prof.get("reviews") != []:
                for rev in prof.get("reviews"):
                    if rev.get("course") is not None and prof.get("type") == "professor":
                        reviews.append({
                            "name": prof.get("name", ""),
                            "slug": prof.get("slug", ""),
                            "type": prof.get("type", "unknown"),
                            "course": rev.get("course", ""),
                            "rating": rev.get("rating", None),
                            "review": rev.get("review", ""),
                            "date": rev.get("created", "")[:10]
                        })
                        count += 1
                        print(count)

        offset += limit

    if reviews:
        df = pd.DataFrame(reviews)
        df = df.sort_values(by=["name", "course"])
        df.to_csv(filename, index=False, encoding="utf-8")
        print(f"Saved {len(df)} reviews to {filename}")
        return df
    else:
        print("No valid review data collected. File not saved.")
        return pd.DataFrame()

# Call the function
fetch_planetterp_reviews()


Loaded existing review data.


,name,slug,type,course,rating,review,date
0,A Anthony,anthony,professor,AMST203,1.0,"By far the worst professor I’ve ever had, and ...",2018-08-17
1,A Kruglanski,kruglanski,professor,PSYC489H,2.0,"DO NOT TAKE PSYC489H ""Motivated Social Cogniti...",2015-09-07
2,A Kruglanski,kruglanski,professor,PSYC489T,5.0,I took PSYC489T in Spring '24. The class meets...,2024-09-24
3,A Sharma,sharma_a,professor,ASTR300,2.0,"Very boring, it's hard to maintain your focus ...",2019-04-04
4,A Sharma,sharma_a,professor,ASTR300,1.0,You'll pass but this class will be the most bo...,2019-05-26
...,...,...,...,...,...,...,...
38109,Zubin Jelveh,jelveh,professor,CCJS418E,2.0,Unless you have a strong passion or interest i...,2025-01-20
38110,Zubin Jelveh,jelveh,professor,INST414,3.0,Professor is very nice and makes an effort to ...,2023-01-21
38111,Zubin Jelveh,jelveh,professor,INST414,4.0,This was my favorite class of the semester. Th...,2023-12-29
38112,Zubin Jelveh,jelveh,professor,INST414,1.0,"If you take this professor, you better pay att...",2024-10-03


### 5. This function, `fetch_planetterp_grades()`, takes a DataFrame of professor reviews, fetches corresponding grade distribution data for each professor from the PlanetTerp API, and stores the results in a CSV file (`PT_grade_data.csv`). If the file already exists with data, it loads it instead of downloading again. Each professor’s course grade records are gathered and returned as a pandas DataFrame.


In [14]:
def fetch_planetterp_grades(fetch_planetterp_reviews):
    """
    Fetches grade data for each professor in review_df and saves it to PT_grade_data.csv.
    Returns a DataFrame.
    """
    filename = "PT_grade_data.csv"

    if os.path.exists(filename) and os.path.getsize(filename) > 0:
        print(" Loaded existing grade data.")
        return pd.read_csv(filename)

    print(" Downloading grade data for professors...")

    base_url = "https://planetterp.com/api/v1/grades"
    professors = review_df["name"].dropna().unique()
    all_grades = []

    for prof in tqdm(professors):
        encoded_name = quote_plus(prof)
        url = f"{base_url}?professor={encoded_name}"

        try:
            response = requests.get(url)
            if response.status_code != 200:
                print(f" HTTP error {response.status_code} for {prof}")
                continue

            data = response.json()

            for record in data:
                all_grades.append({
                    "professor": prof,
                    "course": record.get("course", ""),
                    "semester": record.get("semester", ""),
                    "section": record.get("section", ""),
                    "A+": record.get("A+", 0),
                    "A":  record.get("A", 0),
                    "A-": record.get("A-", 0),
                    "B+": record.get("B+", 0),
                    "B":  record.get("B", 0),
                    "B-": record.get("B-", 0),
                    "C+": record.get("C+", 0),
                    "C":  record.get("C", 0),
                    "C-": record.get("C-", 0),
                    "D+": record.get("D+", 0),
                    "D":  record.get("D", 0),
                    "D-": record.get("D-", 0),
                    "F":  record.get("F", 0),
                    "W":  record.get("W", 0),
                    "Other": record.get("Other", 0)
                })

        except Exception as e:
            print(f" Error for {prof}: {e}")
            continue

    if all_grades:
        df = pd.DataFrame(all_grades)
        df.to_csv(filename, index=False)
        print(f" Saved {len(df)} grade records to {filename}")
        return df
    else:
        print(" No grade data collected. File not saved.")
        return pd.DataFrame()
# Call the function
fetch_planetterp_grades(fetch_planetterp_reviews)

 Loaded existing grade data.


,professor,course,semester,section,A+,A,A-,B+,B,B-,C+,C,C-,D+,D,D-,F,W,Other
0,A Seyed,INST737,201508,101,18,9,0,1,0,0,0,0,0,0,0,0,0,0,0
1,A Seyed,INST737,201601,101,20,5,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A Seyed,INST737,201608,SG01,0,7,1,0,0,1,0,0,0,0,0,0,0,1,0
3,A Seyed,INST737,201701,SG01,0,11,1,0,0,0,0,0,0,0,0,0,0,0,0
4,A Seyed,INST737,201708,101,14,6,0,0,0,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150706,Xiaodong Shi,CHEM889C,202401,101,0,11,0,0,1,0,0,0,0,0,0,0,0,0,0
150707,Belinda Qian He,CHIN315,202308,101,22,2,3,0,1,0,0,1,0,0,0,0,0,0,0
150708,Belinda Qian He,CHIN418T,202308,101,15,1,3,2,0,0,1,0,0,1,1,0,0,0,0
150709,Belinda Qian He,CHIN315,202401,101,12,6,1,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
def fetch_planetterp_grades(fetch_planetterp_reviews):
    """
    Fetches grade data for each professor in review_df and saves it to PT_grade_data.csv.
    Returns a DataFrame.
    """
    filename = "PT_grade_data.csv"

    if os.path.exists(filename) and os.path.getsize(filename) > 0:
        print(" Loaded existing grade data.")
        return pd.read_csv(filename)

    print(" Downloading grade data for professors...")

    base_url = "https://planetterp.com/api/v1/grades"
    professors = review_df["name"].dropna().unique()
    all_grades = []

    for prof in tqdm(professors):
        encoded_name = quote_plus(prof)
        url = f"{base_url}?professor={encoded_name}"

        try:
            response = requests.get(url)
            if response.status_code != 200:
                print(f" HTTP error {response.status_code} for {prof}")
                continue

            data = response.json()

            for record in data:
                all_grades.append({
                    "professor": prof,
                    "course": record.get("course", ""),
                    "semester": record.get("semester", ""),
                    "section": record.get("section", ""),
                    "A+": record.get("A+", 0),
                    "A":  record.get("A", 0),
                    "A-": record.get("A-", 0),
                    "B+": record.get("B+", 0),
                    "B":  record.get("B", 0),
                    "B-": record.get("B-", 0),
                    "C+": record.get("C+", 0),
                    "C":  record.get("C", 0),
                    "C-": record.get("C-", 0),
                    "D+": record.get("D+", 0),
                    "D":  record.get("D", 0),
                    "D-": record.get("D-", 0),
                    "F":  record.get("F", 0),
                    "W":  record.get("W", 0),
                    "Other": record.get("Other", 0)
                })

        except Exception as e:
            print(f" Error for {prof}: {e}")
            continue

    if all_grades:
        df = pd.DataFrame(all_grades)
        df.to_csv(filename, index=False)
        print(f" Saved {len(df)} grade records to {filename}")
        return df
    else:
        print(" No grade data collected. File not saved.")
        return pd.DataFrame()
# Call the function
fetch_planetterp_grades(fetch_planetterp_reviews)

 Loaded existing grade data.


,professor,course,semester,section,A+,A,A-,B+,B,B-,C+,C,C-,D+,D,D-,F,W,Other
0,A Seyed,INST737,201508,101,18,9,0,1,0,0,0,0,0,0,0,0,0,0,0
1,A Seyed,INST737,201601,101,20,5,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A Seyed,INST737,201608,SG01,0,7,1,0,0,1,0,0,0,0,0,0,0,1,0
3,A Seyed,INST737,201701,SG01,0,11,1,0,0,0,0,0,0,0,0,0,0,0,0
4,A Seyed,INST737,201708,101,14,6,0,0,0,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150706,Xiaodong Shi,CHEM889C,202401,101,0,11,0,0,1,0,0,0,0,0,0,0,0,0,0
150707,Belinda Qian He,CHIN315,202308,101,22,2,3,0,1,0,0,1,0,0,0,0,0,0,0
150708,Belinda Qian He,CHIN418T,202308,101,15,1,3,2,0,0,1,0,0,1,1,0,0,0,0
150709,Belinda Qian He,CHIN315,202401,101,12,6,1,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
def combine_diamondback_data(input_dir=r"C:\Users\LENOVO\HTML\TEKHER\SRAPPING\DB_Years", output_file="DB_combined_data.csv"):
    all_dfs = []

    # Step 1: Load all yearly files
    for file in os.listdir(input_dir):
        if file.endswith(".csv") and file.startswith("DB_"):
            df = pd.read_csv(os.path.join(input_dir, file))
            all_dfs.append(df)

    if not all_dfs:
        print(" No Diamondback CSV files found.")
        return

    combined_df = pd.concat(all_dfs, ignore_index=True)

    # Clean and convert salary
    combined_df["Salary"] = combined_df["Salary"].replace(r'[$,]', '', regex=True).astype(float)

    # Standardize name: LAST, FIRST → FIRST LAST
    def standardize(name):
        if "," in name:
            last, first = name.split(",", 1)
            return f"{first.strip().upper()} {last.strip().upper()}"
        return name.strip().upper()

    combined_df["name"] = combined_df["Employee"].apply(standardize)

    # Group by name and year to combine multiple salaries in the same year
    grouped = combined_df.groupby(["name", "Year"]).agg({
        "Salary": "sum",
        "Department": lambda x: list(set(x))
    }).reset_index()

    # Group by name again and collect lists
    grouped_final = grouped.groupby("name").agg({
        "Year": lambda x: list(x),
        "Salary": lambda x: list(x),
        "Department": lambda x: list(x)
    }).reset_index()

    grouped_final.rename(columns={
        "Year": "years_taught",
        "Salary": "salaries",
        "Department": "departments"
    }, inplace=True)

    # ----------  Explode aggregated data ----------
    rows = []
    for _, row in grouped_final.iterrows():
        years = row["years_taught"]
        salaries = row["salaries"]
        departments = row["departments"]

        # Handle misalignment
        length = min(len(years), len(salaries), len(departments))

        for i in range(length):
            rows.append({
                "name": row["name"],
                "year": years[i],
                "salary": salaries[i],
                "department": departments[i]  # or ", ".join(departments[i]) if needed
            })

    exploded_df = pd.DataFrame(rows)

    # Save exploded result
    exploded_df.to_csv(output_file, index=False)
    print(f" Combined and exploded salary data saved to {output_file}")
    print(exploded_df.head())

    return exploded_df


In [17]:
combine_diamondback_data()

 Combined and exploded salary data saved to DB_combined_data.csv
                    name  year    salary department
0         A BETH DESKINS  2013  49605.55     [PRES]
1         A BETH DESKINS  2014  52371.06     [PRES]
2  A F M KAMAL CHOWDHURY  2022  76356.00     [CMNS]
3          A JANE HARRAH  2013  19599.48     [VPAA]
4        A PATRICE SEYED  2015   8572.90     [INFO]


,name,year,salary,department
0,A BETH DESKINS,2013,49605.55,[PRES]
1,A BETH DESKINS,2014,52371.06,[PRES]
2,A F M KAMAL CHOWDHURY,2022,76356.00,[CMNS]
3,A JANE HARRAH,2013,19599.48,[VPAA]
4,A PATRICE SEYED,2015,8572.90,[INFO]
...,...,...,...,...
100546,ZULMA R ESCOBAR,2021,28051.71,[VPAF]
100547,ZULMA R ESCOBAR,2022,35862.77,[VPA-]
100548,ZUYUE DU,2014,30.00,[CMNS]
100549,ZVI SAHAR,2016,20349.91,[ARHU]


In [18]:
# Load the combined salary data
df = pd.read_csv("DB_combined_data.csv")

# Display the first few rows
print(df)


                         name  year    salary department
0              A BETH DESKINS  2013  49605.55   ['PRES']
1              A BETH DESKINS  2014  52371.06   ['PRES']
2       A F M KAMAL CHOWDHURY  2022  76356.00   ['CMNS']
3               A JANE HARRAH  2013  19599.48   ['VPAA']
4             A PATRICE SEYED  2015   8572.90   ['INFO']
...                       ...   ...       ...        ...
100546        ZULMA R ESCOBAR  2021  28051.71   ['VPAF']
100547        ZULMA R ESCOBAR  2022  35862.77   ['VPA-']
100548               ZUYUE DU  2014     30.00   ['CMNS']
100549              ZVI SAHAR  2016  20349.91   ['ARHU']
100550              ZVI SAHAR  2019  20000.00   ['ARHU']

[100551 rows x 4 columns]


In [19]:
# Load files
review_df = pd.read_csv("PT_review_data.csv")
grade_df = pd.read_csv("PT_grade_data.csv")
salary_df = pd.read_csv("DB_combined_data.csv")  # for later joining


In [20]:
# Parse the date
review_df["date"] = pd.to_datetime(review_df["date"], errors='coerce')
review_df["year"] = review_df["date"].dt.year
review_df["month"] = review_df["date"].dt.month

# Map month to season
def get_season(month):
    if month in [1, 2]:
        return "winter"
    elif month in [3, 4, 5]:
        return "spring"
    elif month in [6, 7, 8]:
        return "summer"
    else:
        return "fall"

review_df["season"] = review_df["month"].apply(get_season)

# Group and aggregate
average_reviews_df = (
    review_df
    .groupby(["name", "course", "year", "season"])
    .agg(average_rating=("rating", "mean"), num_reviews=("rating", "count"))
    .reset_index()
)


In [21]:
average_reviews_df

,name,course,year,season,average_rating,num_reviews
0,A Anthony,AMST203,2018,summer,1.0,1
1,A Kruglanski,PSYC489H,2015,fall,2.0,1
2,A Kruglanski,PSYC489T,2024,fall,5.0,1
3,A Sharma,ASTR300,2019,fall,1.0,1
4,A Sharma,ASTR300,2019,spring,1.5,2
...,...,...,...,...,...,...
21341,Zubin Jelveh,CCJS418E,2024,fall,5.0,1
21342,Zubin Jelveh,CCJS418E,2025,winter,2.0,1
21343,Zubin Jelveh,INST414,2023,fall,4.0,1
21344,Zubin Jelveh,INST414,2023,winter,3.0,1


In [22]:
# Extract year and season from semester
grade_df["year"] = grade_df["semester"].astype(str).str[:4].astype(int)
grade_df["season"] = grade_df["semester"].astype(str).str[4:].map({
    "01": "winter",
    "05": "spring",
    "08": "fall"
}).fillna("unknown")

# GPA mapping
gpa_weights = {
    "A+": 4.0, "A": 4.0, "A-": 3.7,
    "B+": 3.3, "B": 3.0, "B-": 2.7,
    "C+": 2.3, "C": 2.0, "C-": 1.7,
    "D+": 1.3, "D": 1.0, "D-": 0.7,
    "F": 0.0
}

# Convert grade columns to numeric
for grade in gpa_weights:
    grade_df[grade] = pd.to_numeric(grade_df[grade], errors="coerce").fillna(0)

grade_df["num_students"] = grade_df[list(gpa_weights)].sum(axis=1)
grade_df["total_points"] = sum(grade_df[g] * w for g, w in gpa_weights.items())
grade_df["average_gpa"] = grade_df["total_points"] / grade_df["num_students"]
grade_df.drop(columns=["total_points"], inplace=True)

# Rename for merge
grade_df.rename(columns={"professor": "name"}, inplace=True)

# Group
average_grades_df = (
    grade_df
    .groupby(["name", "course", "year", "season"])
    .agg(average_gpa=("average_gpa", "mean"), num_students=("num_students", "sum"))
    .reset_index()
)


In [23]:
average_grades_df

,name,course,year,season,average_gpa,num_students
0,A Anthony,AMST202,2016,fall,2.462500,24
1,A Anthony,AMST202,2017,winter,2.934783,23
2,A Anthony,AMST203,2017,fall,2.818718,28
3,A Anthony,AMST203,2018,fall,3.208333,43
4,A Anthony,AMST203,2018,winter,2.476190,21
...,...,...,...,...,...,...
94759,sahar jendi,ARAB210,2012,fall,2.857143,14
94760,sahar jendi,ARAB211,2013,winter,3.820000,5
94761,yh Patt,WMST200,2012,fall,3.210069,34
94762,yh Patt,WMST200,2013,winter,2.891667,12


In [24]:
both_averages_df = pd.merge(
    average_reviews_df,
    average_grades_df,
    on=["name", "course", "year", "season"],
    how="outer"
)


In [25]:
both_averages_df

,name,course,year,season,average_rating,num_reviews,average_gpa,num_students
0,A Anthony,AMST202,2016,fall,NaN,NaN,2.462500,24.0
1,A Anthony,AMST202,2017,winter,NaN,NaN,2.934783,23.0
2,A Anthony,AMST203,2017,fall,NaN,NaN,2.818718,28.0
3,A Anthony,AMST203,2018,fall,NaN,NaN,3.208333,43.0
4,A Anthony,AMST203,2018,summer,1.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...
110692,sahar jendi,ARAB210,2012,fall,NaN,NaN,2.857143,14.0
110693,sahar jendi,ARAB211,2013,winter,NaN,NaN,3.820000,5.0
110694,yh Patt,WMST200,2012,fall,NaN,NaN,3.210069,34.0
110695,yh Patt,WMST200,2013,winter,NaN,NaN,2.891667,12.0


In [26]:
def aggregate_professor(group):
    return group[[
        "name", "course", "year", "season",
        "average_rating", "num_reviews",
        "average_gpa", "num_students"
    ]]
pt_exploded_df = both_averages_df.groupby("name").apply(aggregate_professor).reset_index(drop=True)
pt_exploded_df["name"] = pt_exploded_df["name"].str.upper().str.strip()
pt_exploded_df.to_csv("PT_combined_data.csv", index=False)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_21376\3000729248.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pt_exploded_df = both_averages_df.groupby("name").apply(aggregate_professor).reset_index(drop=True)


In [27]:
df1 = pd.read_csv("PT_combined_data.csv")

# Display the first few rows
df1

,name,course,year,season,average_rating,num_reviews,average_gpa,num_students
0,A ANTHONY,AMST202,2016,fall,NaN,NaN,2.462500,24.0
1,A ANTHONY,AMST202,2017,winter,NaN,NaN,2.934783,23.0
2,A ANTHONY,AMST203,2017,fall,NaN,NaN,2.818718,28.0
3,A ANTHONY,AMST203,2018,fall,NaN,NaN,3.208333,43.0
4,A ANTHONY,AMST203,2018,summer,1.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...
110692,SAHAR JENDI,ARAB210,2012,fall,NaN,NaN,2.857143,14.0
110693,SAHAR JENDI,ARAB211,2013,winter,NaN,NaN,3.820000,5.0
110694,YH PATT,WMST200,2012,fall,NaN,NaN,3.210069,34.0
110695,YH PATT,WMST200,2013,winter,NaN,NaN,2.891667,12.0
